- RBMT : 언어학자 -> SMT : 언어학자를 해고할수록 성능 증가(n-gram)
- seq2seq는 사실 문제가 있음 -> attention 필수(기억력의 문제 보정)
- 단어 시퀀스에 확률을 할당하기 위한 학습 방법이 이전 단어로부터 다음 단어을 예측하는 것.
- seq2seq로 만든 챗봇은 open domain
- 마지막 단어의 hidden state = context
- beam search : 성능에 영향 큼
- 숙제 : 서브클래싱 구현

# BiLingual Evaluation Understudy Score
- BLEU는 기계 번역 결과와 사람이 직접 번역한 결과가 얼마나 유사한지 비교하여 번역에 대한 성능을 측정하는 방법입니다. 측정 기준은 n-gram에 기반합니다.
- 번역된 문장을 `cand`(candidate), 완벽한 번역 문장을 `ref`(Reference)라고 하겠습니다.

In [ ]:
import nltk
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd
import numpy as np
import re
import os
import wget
import urllib
import sentencepiece as sp
import csv
import urllib3
import zipfile
import shutil
from collections import Counter

## 1. 직접 구현하기

In [ ]:
def get_ngram2cnt(cand, n):
    return Counter(nltk.ngrams(cand, n))

In [ ]:
get_ngram2cnt(cand1, 3)

Counter({('it', 'is', 'a'): 1,
         ('is', 'a', 'guide'): 1,
         ('a', 'guide', 'to'): 1,
         ('guide', 'to', 'action'): 1,
         ('to', 'action', 'which'): 1,
         ('action', 'which', 'ensures'): 1,
         ('which', 'ensures', 'that'): 1,
         ('ensures', 'that', 'the'): 1,
         ('that', 'the', 'military'): 1,
         ('the', 'military', 'always'): 1,
         ('military', 'always', 'obeys'): 1,
         ('always', 'obeys', 'the'): 1,
         ('obeys', 'the', 'commands'): 1,
         ('the', 'commands', 'of'): 1,
         ('commands', 'of', 'the'): 1,
         ('of', 'the', 'party'): 1})

### 1-1. N-gram Precision

In [ ]:
def get_ngram_precision(cand, refs, n):
    ngram2cnt_refs = Counter()
    for ref in refs:
        ngram2cnt_refs += get_ngram2cnt(ref, n)
    ngrams_in_refs = 0
    len_cand = 0
    for ngram, cnt in get_ngram2cnt(cand, n).items():
        if ngram in ngram2cnt_refs:
            ngrams_in_refs += cnt 
        len_cand += cnt
    return ngrams_in_refs/len_cand

In [ ]:
print(get_ngram_precision(cand1, refs, 1))
print(get_ngram_precision(cand2, refs, 1))

0.9444444444444444
0.5714285714285714


### 1-2. Modified N-gram Precision

In [ ]:
def get_modified_ngram_precision(cand, refs, n):
    def get_count_clip(ngram, cand, refs, n):
        def get_max_ref_count(ngram, refs, n):
            temp = list()
            for ref in refs:
                ngram2cnt_ref = get_ngram2cnt(ref, n)
                temp.append(ngram2cnt_ref[ngram])
            return max(temp)    

        def get_count(ngram, cand, n):
            return get_ngram2cnt(cand, 1)[ngram]

        return min(get_count(ngram, cand, n), get_max_ref_count(ngram, refs, n))
    
    sum_countclip = 0
    len_cand = 0
    for ngram, cnt in get_ngram2cnt(cand, n).items():
        sum_countclip += get_count_clip(ngram, cand, refs, n)
        len_cand += cnt
    return sum_countclip/len_cand

In [ ]:
print(get_modified_ngram_precision(cand1, refs, 1))
print(get_modified_ngram_precision(cand2, refs, 1))

0.9444444444444444
0.5714285714285714


In [ ]:
cand = "the the the the the the the"
print(get_ngram_precision(cand.split(" "), refs, 1))
print(get_modified_ngram_precision(cand.split(" "), refs, 1))

1.0
0.5714285714285714


### 1-3. 짧은 문장 길이에 대한 패널티(Brevity Penalty)

Ref가 1개라면 Ca와 Ref의 두 문장의 길이만을 가지고 계산하면 되겠지만 여기서는 Ref가 여러 개일 때를 가정하고 있으므로 r은 "모든 Ref들 중에서 Ca와 가장 길이 차이가 작은 Ref의 길이"로 합니다. r을 구하는 코드는 아래와 같습니다.

In [ ]:
def closest_ref_length(cand, ref_list): # Ca 길이와 가장 근접한 Ref의 길이를 리턴하는 함수
    ca_len = len(cand) # ca 길이
    ref_lens = (len(ref) for ref in ref_list) # Ref들의 길이
    closest_ref_len = min(ref_lens, key=lambda ref_len: (abs(ref_len - ca_len), ref_len))
    # 길이 차이를 최소화하는 Ref를 찾아서 Ref의 길이를 리턴
    return closest_ref_len

만약 Ca와 길이가 정확히 동일한 Ref가 있다면 길이 차이가 0인 최고 수준의 매치(best match length)입니다. 또한 만약 서로 다른 길이의 Ref이지만 Ca와 길이 차이가 동일한 경우에는 더 작은 길이의 Ref를 택합니다. 예를 들어 Ca가 길이가 10인데, Ref 1, 2가 각각 9와 11이라면 길이 차이는 동일하게 1밖에 나지 않지만 9를 택합니다. closest_ref_length 함수를 통해 r을 구했다면, 이제 BP를 구하는 함수 brevity_penalty를 구현해봅시다.

In [ ]:
def brevity_penalty(cand, ref_list):
    ca_len = len(cand)
    ref_len = closest_ref_length(cand, ref_list)

    if ca_len > ref_len:
        return 1
    elif ca_len == 0 :
    # cand가 비어있다면 BP = 0 → BLEU = 0.0
        return 0
    else:
        return np.exp(1 - ref_len/ca_len)

## BLEU 함수 구현하기

이제 최종적으로 BLEU 점수를 계산하는 함수 bleu_score를 구현해봅시다.

In [ ]:
def bleu_score(cand, ref_list, weights=[0.25, 0.25, 0.25, 0.25]):
    bp = brevity_penalty(cand, ref_list) # 브레버티 패널티, BP

    p_n = [modified_precision(cand, ref_list, n=n) for n, _ in enumerate(weights,start=1)] 
    #p1, p2, p3, ..., pn
    score = np.sum([w_i * np.log(p_i) if p_i != 0 else 0 for w_i, p_i in zip(weights, p_n)])
    return bp * np.exp(score)

위 함수가 동작하기 위해서는 앞서 구현한 get_ngram2cnt, count_clip, modified_precision, brevity_penalty 4개의 함수 또한 모두 구현되어져 있어야 합니다. 지금까지 구현한 BLEU 코드로 계산된 점수와 NLTK 패키지에 이미 구현되어져 있는 BLEU 코드로 계산된 점수를 비교해봅시다.

## 2. nltk.translate.bleu_score.sentence_bleu()

In [ ]:
ref = [["this", "is", "a", "test"], ["this", "is" "test"]]
cand = ["this", "is", "a", "test"]
score = nltk.translate.bleu_score.sentence_bleu(ref, cand)
print(score)

1.0


In [ ]:
refs = [[["this", "is", "a", "test"], ["this", "is" "test"]]]
cands = [["this", "is", "a", "test"]]
score = nltk.translate.bleu_score.corpus_bleu(refs, cands)
print(score)

1.0


- 각 N-gram(N = 1, 2, 3, 4)에 대해 가중치를 서로 다르게 설정해 weighted geometric mean을 구할 수도 있습니다.
- 몇 개의 N-gram을 사용하느냐에 따라 BLEU-1, BLEU-2, BLEU-3, BLEU-4라고 부릅니다.

In [ ]:
# cumulative BLEU scores
ref = [["this", "is", "small", "test"]]
cand = ["this", "is", "a", "test"]

print(f"BLEU-2 = {nltk.translate.bleu_score.sentence_bleu(ref, cand, weights=(1/2, 1/2, 0, 0))}")

BLEU-2 = 0.49999999999999994


C:\Users\5CG7092POZ\Anaconda3\envs\tf2.3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\5CG7092POZ\Anaconda3\envs\tf2.3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


## NLTK의 BLEU Vs. 구현한 BLEU 함수

In [ ]:
import nltk.translate.bleu_score as bleu


cand = "It is a guide to action which ensures that the military always obeys the commands of the party"
refs = [
    "It is a guide to action that ensures that the military will forever heed Party commands",
    "It is the guiding principle which guarantees the military forces always being under the command of the Party",
    "It is the practical guide for the army always to heed the directions of the party"
]

# 이번 챕터에서 구현한 코드로 계산한 BLEU 점수
print(bleu_score(cand.split(),list(map(lambda ref: ref.split(), refs))))
# NLTK 패키지 구현되어져 있는 코드로 계산한 BLEU 점수
print(bleu.nltk.translate.bleu_score.sentence_bleu(list(map(lambda ref: ref.split(), refs)),cand.split()))

0.5045666840058485
0.5045666840058485
